# Testing new parameters on 1,000 maps through TI9

Testing different logistic scale and time scale (for covariance function) parameter values using AUC and likelihood.

## Inputs

### Input files

In [2]:
MATCHES_DF = 'data/raw/premium_matches.2019-08-31.json.gz'

In [3]:
from collections import OrderedDict
files = OrderedDict()
for ls in [0.5, 1, 2, 3, 4.5, 6, 8]:
    files['ls = ' + str(ls)] = (
        ls,  # Logistic scale
        f'fitted/ti9_full_new_params_by_series.ls-{ls}.win_probs.gz',
        f'fitted/ti9_full_new_params_by_series.ls-{ls}.player_skills'
            '.gz'
    )
for ts in [0.5, 0.75, 1, 1.25, 1.5, 2, 3]:
    files['ts = ' + str(ts)] = (
        3.0,  # Logistic scale
        f'fitted/ti9_full_new_params_by_series.ts-{ts}.win_probs.gz',
        f'fitted/ti9_full_new_params_by_series.ts-{ts}.player_skills.gz'
    )
files

OrderedDict([('ls = 0.5',
              (0.5,
               'fitted/ti9_full_new_params_by_series.ls-0.5.win_probs.gz',
               'fitted/ti9_full_new_params_by_series.ls-0.5.player_skills.gz')),
             ('ls = 1',
              (1,
               'fitted/ti9_full_new_params_by_series.ls-1.win_probs.gz',
               'fitted/ti9_full_new_params_by_series.ls-1.player_skills.gz')),
             ('ls = 2',
              (2,
               'fitted/ti9_full_new_params_by_series.ls-2.win_probs.gz',
               'fitted/ti9_full_new_params_by_series.ls-2.player_skills.gz')),
             ('ls = 3',
              (3,
               'fitted/ti9_full_new_params_by_series.ls-3.win_probs.gz',
               'fitted/ti9_full_new_params_by_series.ls-3.player_skills.gz')),
             ('ls = 4.5',
              (4.5,
               'fitted/ti9_full_new_params_by_series.ls-4.5.win_probs.gz',
               'fitted/ti9_full_new_params_by_series.ls-4.5.player_skills.gz')),
             (

### Parameters

In [4]:
# Scaling factor for logistic regression win probability.
# LOGISTIC_SCALE = 3.0

# Maximum difference in minutes between the Fairlay closing datetime and the
# Datdota map start datetime.
MAX_TIMEDIFF_MINS = 1440

## Setup.

### Load libraries.

In [5]:
import os
while 'src' not in os.listdir():
    os.chdir("..")

In [6]:
import gzip
import json
import pickle

import numpy as np
import pandas as pd
import scipy

import src.fairlay
import src.load
import src.statics
import src.stats

### Load data.

#### Matches and predictions

In [7]:
with gzip.open(MATCHES_DF, 'rb') as fh:
    matches = src.load.MatchDF.from_json(json.load(fh)['data'])

In [8]:
match_pred_objs = OrderedDict()
for label, val in files.items():
    logistic_scale, pred_file, skills_file = val
    pred_df = pd.read_csv(pred_file, header=0, sep='\t', index_col=0)
    skills_mat = pd.read_csv(skills_file, header=0, sep='\t', index_col=0)
    skills_mat.columns = skills_mat.columns.astype(int)
    match_pred_objs[label] = \
        src.stats.MatchPred(matches, pred_df, logistic_scale, skills_mat)

## AUCs

In [17]:
aucs = []
for label, match_pred in match_pred_objs.items():
    auc = match_pred._roc_auc(
        False,
        match_pred.df.iter > 0)
    aucs.append(auc)

#     print("--------")
#     print(label)
#     print("TI9 group stage AUC: {:.3f}".format(ti9_group_stage_auc))
#     print("TI9 playoffs AUC: {:.3f}".format(ti9_playoffs_auc))

pd.Series(aucs, index=match_pred_objs.keys())

ls = 0.5     0.645280
ls = 1       0.658237
ls = 2       0.667028
ls = 3       0.669351
ls = 4.5     0.667513
ls = 6       0.663592
ls = 8       0.658513
ts = 0.5     0.663824
ts = 0.75    0.666740
ts = 1       0.668550
ts = 1.25    0.669351
ts = 1.5     0.670185
ts = 2       0.670513
ts = 3       0.669688
dtype: float64

## Likelihoods

In [20]:
logliks = []
for label, match_pred in match_pred_objs.items():
    loglik = sum(match_pred.match_loglik(False)[match_pred.df.iter > 0])
    logliks.append(loglik)
pd.Series(logliks, index=match_pred_objs.keys())

ls = 0.5    -743.879784
ls = 1      -681.845221
ls = 2      -653.947995
ls = 3      -649.853927
ls = 4.5    -651.762285
ls = 6      -656.031617
ls = 8      -662.020677
ts = 0.5    -652.936902
ts = 0.75   -651.012802
ts = 1      -650.182926
ts = 1.25   -649.853927
ts = 1.5    -649.547137
ts = 2      -649.294076
ts = 3      -649.336656
dtype: float64